In [1]:
from pathlib import Path
import json
import datetime as dt
from itertools import product
import math
import sys

import googlemaps
import numpy as np
import pandas as pd
import geopandas as gpd

sys.path.append('../')

import googlemaps_extras as ge

ROOT = Path('../')
DATA_DIR = ROOT/'tests'/'data'

%load_ext autoreload
%autoreload 2

In [5]:
# Test some
path = DATA_DIR/'points.geojson'
points = gpd.read_file(str(path))

origs = [p.coords[0] for p in points['geometry'][:2]]
dests = [p.coords[0] for p in points['geometry'][2:]]
time = dt.datetime(2017, 6, 15, 8)
time.strftime('%Y-%m-%dT%H:%M:%S')
r = client.distance_matrix(ge.flip_coords(origs), ge.flip_coords(dests), mode='driving', 
  traffic_model='pessimistic', departure_time='now')
r
ge.to_df(r, ['bingo', 'bongo', 'boom'], ['woop', 'boop'])

,origin_address,destination_address,duration,distance,origin_id,destination_id
0,"9 Waverley Ave, Glenfield, Auckland 0629, New ...","26 Ben Nevis Pl, Northpark, Auckland 2013, New...",4777,31495,bingo,woop
1,"919 Mount Eden Rd, Mount Eden, Auckland 1024, ...","26 Ben Nevis Pl, Northpark, Auckland 2013, New...",3258,18067,bongo,woop


In [8]:

# Test some
path = DATA_DIR/'points.geojson'
points = gpd.read_file(str(path))
points
ge.build_distance_matrix_df(client, points, points)



hello {"origin_addresses": ["9 Waverley Ave, Glenfield, Auckland 0629, New Zealand"], "destination_addresses": ["919 Mount Eden Rd, Mount Eden, Auckland 1024, New Zealand", "26 Ben Nevis Pl, Northpark, Auckland 2013, New Zealand"], "status": "OK", "rows": [{"elements": [{"status": "OK", "distance": {"value": 18688, "text": "18.7 km"}, "duration": {"value": 1562, "text": "26 mins"}}, {"status": "OK", "distance": {"value": 31495, "text": "31.5 km"}, "duration": {"value": 2304, "text": "38 mins"}}]}]}
hello {"origin_addresses": ["919 Mount Eden Rd, Mount Eden, Auckland 1024, New Zealand"], "destination_addresses": ["9 Waverley Ave, Glenfield, Auckland 0629, New Zealand", "26 Ben Nevis Pl, Northpark, Auckland 2013, New Zealand"], "status": "OK", "rows": [{"elements": [{"status": "OK", "distance": {"value": 18226, "text": "18.2 km"}, "duration": {"value": 1590, "text": "27 mins"}}, {"status": "OK", "distance": {"value": 18597, "text": "18.6 km"}, "duration": {"value": 1739, "text": "29 mins

,origin_address,destination_address,origin_id,destination_id,duration,distance
0,"9 Waverley Ave, Glenfield, Auckland 0629, New ...","919 Mount Eden Rd, Mount Eden, Auckland 1024, ...",s0,s1,1562,18688
1,"9 Waverley Ave, Glenfield, Auckland 0629, New ...","26 Ben Nevis Pl, Northpark, Auckland 2013, New...",s0,s2,2304,31495
0,"919 Mount Eden Rd, Mount Eden, Auckland 1024, ...","9 Waverley Ave, Glenfield, Auckland 0629, New ...",s1,s0,1590,18226
1,"919 Mount Eden Rd, Mount Eden, Auckland 1024, ...","26 Ben Nevis Pl, Northpark, Auckland 2013, New...",s1,s2,1739,18597
0,"26 Ben Nevis Pl, Northpark, Auckland 2013, New...","9 Waverley Ave, Glenfield, Auckland 0629, New ...",s2,s0,2375,31168
1,"26 Ben Nevis Pl, Northpark, Auckland 2013, New...","919 Mount Eden Rd, Mount Eden, Auckland 1024, ...",s2,s1,1719,18567


In [ ]:
def compute_google_cost(n, with_freebies=False):
    """
    Estimate the cost of Google matrix query with n elements at 0.5/1000 USD/element 
    beyond 2500 elements.
    If ``with_freebies``, then treat the first 2500 elements as free.
    """
    d = OrderedDict()
    d['#elements needed'] = n
    d['exceeds 100000-element daily limit?'] = n > 100000 
    d['duration for job in minutes'] = (n/100)/60
    if with_freebies:
        d['cost for job in USD'] = (n - 2500)*(0.5/1000)
    else:
        d['cost for job in USD'] = n*(0.5/1000)
    
    return pd.Series(d)

def df_to_gdf(f, lon_col='lon', lat_col='lat'):
    """
    Given a DataFrame with WGS84 longitude values in the column ``lon_col``
    and WGS84 latitude values in the column ``lat_col``, convert it into
    a GeoDataFrame with a geometry column corresponding to the longitude-latitude points.
    Delete the original longitude and latitude columns, and return the result.
    """
    f['geometry'] = f[[lon_col, lat_col]].apply(lambda p: sg.Point(p), axis=1)
    f = f.drop([lon_col, lat_col], axis=1)
    f = gpd.GeoDataFrame(f)
    return f 
